In [ ]:
pip install "dask[complete]" #package required to use dask data frames. Colab does not support the package directly

     |████████████████████████████████| 830 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 38.8 MB/s 
     |████████████████████████████████| 822 kB 45.1 MB/s 
     |████████████████████████████████| 802 kB 42.2 MB/s 
     |████████████████████████████████| 802 kB 37.4 MB/s 
     |████████████████████████████████| 793 kB 49.9 MB/s 
     |████████████████████████████████| 793 kB 42.1 MB/s 
     |████████████████████████████████| 791 kB 49.9 MB/s 
     |████████████████████████████████| 786 kB 45.4 MB/s 
     |████████████████████████████████| 779 kB 39.8 MB/s 
     |████████████████████████████████| 778 kB 44.8 MB/s 
     |████████████████████████████████| 776 kB 44.9 MB/s 
     |████████████████████████████████| 769 kB 35.9 MB/s 
     |████████████████████████████████| 766 kB 42.6 MB/s 
     |████████████████████████████████| 1.0 MB 43.7 MB/s 
     |████████████████████████████████| 722 kB 53.0 MB/s 
     |████████████████████████████████| 722 kB 39.6 MB/s 
     |█████████

Importing the required libraries

In [ ]:
import dask
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv1D
from tensorflow.keras.layers import MaxPool2D, MaxPool1D
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
import time
from numpy.random import seed
import graphviz
import tensorflow as tf


In [ ]:
from google.colab import drive # Mount the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive #set the working directory to read the required file

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [ ]:
col = ['Transaction_unique_identifier', 'Price', 'Date_of_Transfer', 'Postcode', 'Property_Type', 'Old_New', 
       'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Category_Type', 
       'Record_Status_monthly_file'] # setting up headers for the data
file = dd.read_csv('pp-complete.csv', names= col) #reading the file
file

,Transaction_unique_identifier,Price,Date_of_Transfer,Postcode,Property_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Category_Type,Record_Status_monthly_file
npartitions=73,,,,,,,,,,,,,,,,
,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Steps for preprocessing

In [ ]:
file['Date_of_Transfer'] = dd.to_datetime(file['Date_of_Transfer'])
file['Month'] = file['Date_of_Transfer'].dt.month
file['Year'] = file['Date_of_Transfer'].dt.year
file = file.drop(columns=['SAON','Locality','Street','Transaction_unique_identifier',"Record_Status_monthly_file"],axis=1)
file =file.dropna()
file =file.drop(['Date_of_Transfer','PAON','Postcode'],axis=1)

file['Month'] -= 1  #scaling the months
file["Year"] -= 1995  #scaling the years

In [ ]:
file = file.compute()
file

,Price,Property_Type,Old_New,Duration,Town_City,District,County,PPD_Category_Type,Month,Year
0,18500,F,N,L,TORQUAY,TORBAY,TORBAY,A,0,0
1,73450,D,Y,F,LIVERPOOL,KNOWSLEY,MERSEYSIDE,A,9,0
2,59000,D,N,F,POOLE,POOLE,POOLE,A,2,0
3,31000,D,Y,F,WOODBRIDGE,SUFFOLK COASTAL,SUFFOLK,A,11,0
4,95000,D,N,F,LICHFIELD,LICHFIELD,STAFFORDSHIRE,A,8,0
...,...,...,...,...,...,...,...,...,...,...
308403,51500,O,N,F,LONDON,ISLINGTON,GREATER LONDON,B,7,26
308404,230000,O,N,F,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,B,4,26
308405,205000,F,N,L,CHELMSFORD,CHELMSFORD,ESSEX,B,3,26
308406,185000,F,N,L,COLCHESTER,TENDRING,ESSEX,B,0,26


Fitting the models for every county

In [ ]:
grouped_data = file.groupby(['County']) # grouping the available data based on county

In [ ]:
def one_hot_encode (data , prefixes): # Code to perform one-hot encoding
  for prefix in prefixes:
    data = pd.concat([
            data, 
            pd.get_dummies(
                data[prefix], prefix=prefix,
                drop_first=True, sparse=False
            )
        ],
        axis=1
    )
    data = data.drop(prefix, axis=1)
  return data

In [ ]:

def get_model(cols): # Code to build deep neural network for every county.

  # Create the model: model

  model = Sequential()



  # Add the first hidden layer

  model.add(Dense(1000,activation='relu',input_shape =(cols,)))



  # Add the second hidden layer

  model.add(Dense(800,activation='relu'))



  model.add(Dense(1000,activation='relu'))



  model.add(Dense(800,activation='relu'))

  model.add(Dropout(0.10))

  model.add(Dense(1000,activation='relu'))

  model.add(Dropout(0.10))

  model.add(Dense(800,activation='relu'))

  model.add(Dropout(0.10))



  model.add(Dense(1000,activation='relu'))

  model.add(Dense(800,activation='relu'))



  # Add the output layer

  model.add(Dense(1,activation='relu'))



# Compile the model

  model.compile(optimizer='adam',loss='mean_absolute_percentage_error')

  return model


In [ ]:
def remove_outliers(df1_scaled): #function to remove outliers for training

    df1_no_out = df1_scaled.copy()

    df1_no_out.drop(df1_no_out[df1_no_out['Price']>3].index,inplace=True)

    return df1_no_out




def standardize(df): # funciton to standardize the price column

    temp=df.copy()

    temp['Price']= (temp['Price']-temp['Price'].mean())/temp['Price'].std()

    return temp



def destandardize(df1,df2): # function to de-standardize th eprice column

    temp = df1.copy()

    temp['Price'] = (temp['Price']*df2['Price'].std())+df2['Price'].mean()

    return temp

In [ ]:
lr_models = {}
dnn_models = {}
prefixes = [ 'Property_Type', 'Old_New', 
       'Duration', 'Town_City', 'District', 'PPD_Category_Type']
seed = 0
r2 = [] # storing the performance of the models on different metric
mse = []
mape = []
mae = []

In [ ]:

for name,data in grouped_data: # perform training for each county

    data= data.drop('County',axis =1) #County column is now not needed for training

    standardized_data = standardize(data)

    standardized_data = remove_outliers(standardized_data)

    destandardized_data = destandardize(standardized_data,data)

    destandardized_data = one_hot_encode(data,prefixes)

    y = destandardized_data['Price']

    X = destandardized_data.drop('Price', axis =1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed) #validation split size is set to 20%

    LR = ElasticNet(alpha=1, l1_ratio=0.5) # linear regression which penalizes model for L1 and L2

    LR.fit(X_train,y_train)#train the model

    lr_models[name] = LR

    y_pred = LR.predict(X_test) # storing the predictions for made by the linear regression model

    v = r2_score(y_test, y_pred) # validating performance on various metrics
    r2.append(v)
    print('R2 for ',name, ' ->', v)

    v = mean_squared_error(y_test, y_pred)
    mse.append(v)
    print('MSE for ',name, ' ->', v)

    v = mean_absolute_percentage_error(y_test, y_pred)
    mape.append(v)
    print('MAPE for ',name, ' ->', v)

    v = mean_absolute_error(y_test, y_pred)
    mae.append(v)
    print('MAE for ',name, ' ->',v)
    print("**********************")

R2 for  AVON  -> 0.0802419970343099
MSE for  AVON  -> 1588620378.9612598
MAPE for  AVON  -> 0.4528385093077556
MAE for  AVON  -> 24448.14441659894
**********************
R2 for  BATH AND NORTH EAST SOMERSET  -> 0.0843331299574489
MSE for  BATH AND NORTH EAST SOMERSET  -> 189464765689.81656
MAPE for  BATH AND NORTH EAST SOMERSET  -> 25.192539382607848
MAE for  BATH AND NORTH EAST SOMERSET  -> 130708.15611383769
**********************
R2 for  BEDFORD  -> 0.036373765944672676
MSE for  BEDFORD  -> 219937987314.71048
MAPE for  BEDFORD  -> 0.5892694028271868
MAE for  BEDFORD  -> 109211.77224379346
**********************
R2 for  BEDFORDSHIRE  -> 0.3751737112350837
MSE for  BEDFORDSHIRE  -> 5648208877.328908
MAPE for  BEDFORDSHIRE  -> 0.4228064045832845
MAE for  BEDFORDSHIRE  -> 47505.16965980934
**********************
R2 for  BERKSHIRE  -> 0.12310166037099268
MSE for  BERKSHIRE  -> 3260096623.6921735
MAPE for  BERKSHIRE  -> 0.40668299672540387
MAE for  BERKSHIRE  -> 36374.42300032094
********

For neural networks, to avoid overfitting, we made use of callbacks. The callback saves the model on a state where minimum validation loss is observed. This model is then used to make the final prediction.

In [ ]:
for name,data in grouped_data: #training for neural networks

    data= data.drop('County',axis =1)

    standardized_data = standardize(data)

    standardized_data = remove_outliers(standardized_data)

    destandardized_data = destandardize(standardized_data,data)

    destandardized_data = one_hot_encode(data,prefixes)

    destandardized_data = one_hot_encode(data,prefixes)

    y = destandardized_data['Price']

    X = destandardized_data.drop('Price', axis =1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

    model = get_model(X.shape[1]) #instantiate a different neural network for each county

    path = '/content/drive/MyDrive/'+name+'.hdf5' #path where the model with minumum validation loss is stored

    checkpoint = ModelCheckpoint(path,monitor='val_loss',save_best_only=True) 

    model.fit(X_train,y_train,validation_split=0.2,epochs = 10,batch_size= 5000, verbose=0, callbacks = [checkpoint])



    dnn_models[name] = model

    model = load_model(path)

    print('for ',name,' ->')

    model.evaluate(X_test,y_test)

    print('*********************************************')


for  AVON  ->
52/52 [==============================] - 0s 2ms/step - loss: 41.3458
*********************************************
for  BATH AND NORTH EAST SOMERSET  ->
261/261 [==============================] - 1s 2ms/step - loss: 74.9270
*********************************************
for  BEDFORD  ->
115/115 [==============================] - 0s 2ms/step - loss: 33.5012
*********************************************
for  BEDFORDSHIRE  ->
404/404 [==============================] - 1s 2ms/step - loss: 22.7738
*********************************************
for  BERKSHIRE  ->
29/29 [==============================] - 0s 3ms/step - loss: 45.8868
*********************************************
for  BLACKBURN WITH DARWEN  ->
179/179 [==============================] - 1s 2ms/step - loss: 47.5827
*********************************************
for  BLACKPOOL  ->
245/245 [==============================] - 1s 2ms/step - loss: 32.7479
*********************************************
for  BLAENAU GWENT  ->
72

In [ ]:
values = {'R2':r2,'MSE':mse, 'MAPE':mape, "MAE":mae} #storing the results obtained in a  pandas dataframe
data = pd.DataFrame(values)
data

,R2,MSE,MAPE,MAE
0,0.080242,1.588620e+09,0.452839,24448.144417
1,0.084333,1.894648e+11,25.192539,130708.156114
2,0.036374,2.199380e+11,0.589269,109211.772244
3,0.375174,5.648209e+09,0.422806,47505.169660
4,0.123102,3.260097e+09,0.406683,36374.423000
...,...,...,...,...
125,0.196373,7.694217e+10,0.449864,99579.315473
126,0.088375,7.806794e+10,0.730319,75740.816630
127,0.116732,2.430059e+10,0.594301,55518.801190
128,0.199767,9.981091e+09,0.636963,53598.232628


In [ ]:
data.to_csv('lr_results.csv') # writing the results to a csv file.